In [13]:
from PIL import Image
from src.color import Color
from rembg import remove
# from octree_quantizer import OctreeQuantizer

In [10]:
REMOVE_BACKGROUND = False

## Remove background from images

In [11]:
# Remove backgroud from image
if (REMOVE_BACKGROUND):
    input_path = './dataset/bg/1.jpg'
    output_path = './dateset/no_bg/1.png'

    input = Image.open(input_path)
    output = remove(input)
    output.save(output_path)

## Load image to octree

In [14]:
image = Image.open('./dataset/no_bg/1.png')
pixels = image.load()
width, height = image.size

# add colors to the octree
for j in range(height):
    for i in range(width):
        Color(*pixels[i, j])